In [ ]:
#

In [ ]:
#Create test set annoy file to use for evaluation metrics


In [ ]:
from helper import *

# Function to return mean reciprocal rank from the model for the given list of question ids and associated list of duplicate "gold standard" question ids
# The score for an item in the question_ids list will be 1 if the first result given by the model matches the corresponding duplicate_ids item
# Searches k top results for each question
# Uses the basic BERT embedding from the helper.py file
def get_mrr_and_accuracy(question_ids, duplicate_ids, k, annoy_test_set_filename):
    mrr = 0.0
    accuracy = 0.0
    num_samples = len(questions)

    for i in num_samples:
        id = question_ids[i]
        duplicate_id = duplicate_ids[i]
        
        question = get_question_body_from_id(id)
        question_embedding = get_paragraph_embedding_bert(question)

        u = AnnoyIndex(768, 'angular')
        u.load(annoy_test_set_filename)
        nearest_i = u.get_nns_by_vector(question_embedding, k) # will return the index of the k nearest neighbor

        answer_rank = 0
        try:
            answer_rank = nearest_i.index(duplicate_id)
        except ValueError:
            # skip rest of iteration and don't update scores if the answer isn't found in the results
            continue
        
        mrr += 1.0 / answer_rank
        accuracy += 1 if answer_rank == 1.0 else 0
        
    mrr /= num_samples
    accuracy /= num_samples
    
    return mrr, accuracy
        

In [ ]:
# set a seed 
# t.set_seed(487)
# we need this to tell what the original sentences are 

# t.build(10) # build 10 trees, allegedly the more the better  
# t.save('bert_embedding_unnormalize.ann')

# Find k-nearest neighbor
k = 3 # first set k = 5
test_sentence = "Why did Old English lose both thorn and eth?" # one of the questions on linguistic stack exchange
test_sentence_vector = encode_sentence(test_sentence)

f = 768
u = AnnoyIndex(f, 'angular')
u.load('bert_embedding1.ann') 
nearest_i = u.get_nns_by_vector(test_sentence_vector, k) # will return the index of the k nearest neighbor
nearest_word = [(i, questions[i]) for i in nearest_i] # 
nearest_word